In [1]:
import os
import gc
import torch
from random import randint
from utils.loss_utils import l1_loss,  ssim
from gaussian_renderer import render
from scene import Scene, GaussianModel
import uuid
from tqdm import tqdm
from utils.image_utils import psnr
from argparse import  Namespace
import json
from arguments import ModelParams, PipelineParams
import os
import sys
import torch
from random import randint
from utils.loss_utils import l1_loss,  ssim
from gaussian_renderer import render
from scene import Scene
import uuid
from tqdm import tqdm
from utils.image_utils import psnr
from argparse import  Namespace
import gc
import json
import os
import time
import uuid
from argparse import ArgumentParser, Namespace
from os import path
from shutil import copyfile
from typing import Dict, Tuple

import torch
from tqdm import tqdm
import nvtx

# %%
from arguments import (
    CompressionParams,
    ModelParams,
    OptimizationParams,
    PipelineParams,
    get_combined_args,
)
from gaussian_renderer import GaussianModel, render
from lpipsPyTorch import lpips
from scene import Scene
from utils.image_utils import psnr
from utils.loss_utils import ssim
import scene.diff_idx as diff_idx
import torchvision

from dataclasses import dataclass
import math
import time
import torch
from torch import nn
from torch_scatter import scatter
from typing import Tuple, Optional
from tqdm import trange
import gc
from scene.gaussian_model import GaussianModel
from utils.splats import to_full_cov, extract_rot_scale
from weighted_distance._C import weightedDistance


In [2]:
class VectorQuantize(nn.Module):
    def __init__(
        self,
        channels: int,
        codebook_size: int = 2**12,
        decay: float = 0.5,
    ) -> None:
        super().__init__()
        self.decay = decay
        self.codebook = nn.Parameter(
            torch.empty(codebook_size, channels), requires_grad=False
        )
        nn.init.kaiming_uniform_(self.codebook)
        self.entry_importance = nn.Parameter(
            torch.zeros(codebook_size), requires_grad=False
        )
        self.eps = 1e-5

    def uniform_init(self, x: torch.Tensor):
        amin, amax = x.aminmax()
        self.codebook.data = torch.rand_like(self.codebook) * (amax - amin) + amin

    def update(self, x: torch.Tensor, importance: torch.Tensor) -> torch.Tensor:
        with torch.no_grad():
            min_dists, idx = weightedDistance(x.detach(), self.codebook.detach())
            acc_importance = scatter(
                importance, idx, 0, reduce="sum", dim_size=self.codebook.shape[0]
            )

            ema_inplace(self.entry_importance, acc_importance, self.decay)

            codebook = scatter(
                x * importance[:, None],
                idx,
                0,
                reduce="sum",
                dim_size=self.codebook.shape[0],
            )

            ema_inplace(
                self.codebook,
                codebook / (acc_importance[:, None] + self.eps),
                self.decay,
            )

            return min_dists

    def forward(
        self,
        x: torch.Tensor,
        return_dists: bool = False,
    ) -> Tuple[torch.Tensor, torch.Tensor, Optional[torch.Tensor]]:
        min_dists, idx = weightedDistance(x.detach(), self.codebook.detach())
        if return_dists:
            return self.codebook[idx], idx, min_dists
        else:
            return self.codebook[idx], idx


def ema_inplace(moving_avg: torch.Tensor, new: torch.Tensor, decay: float):
    moving_avg.data.mul_(decay).add_(new, alpha=(1 - decay))


def vq_features(
    features: torch.Tensor,
    importance: torch.Tensor,
    codebook_size: int,
    vq_chunk: int = 2**16,
    steps: int = 1000,
    decay: float = 0.8,
    scale_normalize: bool = False,
) -> Tuple[torch.Tensor, torch.Tensor]:
    importance_n = importance/importance.max()
    vq_model = VectorQuantize(
        channels=features.shape[-1],
        codebook_size=codebook_size,
        decay=decay,
    ).to(device=features.device)

    vq_model.uniform_init(features)

    errors = []
    for i in trange(steps):
        batch = torch.randint(low=0, high=features.shape[0], size=[vq_chunk])
        vq_feature = features[batch]
        error = vq_model.update(vq_feature, importance=importance_n[batch]).mean().item()
        errors.append(error)
        if scale_normalize:
            # this computes the trace of the codebook covariance matrices
            # we devide by the trace to ensure that matrices have normalized eigenvalues / scales
            tr = vq_model.codebook[:, [0, 3, 5]].sum(-1)
            vq_model.codebook /= tr[:, None]

    gc.collect()
    torch.cuda.empty_cache()

    start = time.time()
    _, vq_indices = vq_model(features)
    torch.cuda.synchronize(device=vq_indices.device)
    end = time.time()
    print(f"calculating indices took {end-start} seconds ")
    return vq_model.codebook.data.detach(), vq_indices.detach()


def join_features(
    all_features: torch.Tensor,
    keep_mask: torch.Tensor,
    codebook: torch.Tensor,
    codebook_indices: torch.Tensor,
) -> Tuple[torch.Tensor, torch.Tensor]:
    keep_features = all_features[keep_mask]
    compressed_features = torch.cat([codebook, keep_features], 0)

    indices = torch.zeros(
        len(all_features), dtype=torch.long, device=all_features.device
    )
    indices[~keep_mask] = codebook_indices
    indices[keep_mask] = torch.arange(len(keep_features), device=indices.device) + len(
        codebook
    )

    return compressed_features, indices


@dataclass
class CompressionSettings:
    codebook_size: int
    importance_prune: float
    importance_include: float
    steps: int
    decay: float
    batch_size: int


def compress_color(
    gaussians: GaussianModel,
    color_importance: torch.Tensor,
    color_comp: CompressionSettings,
    color_compress_non_dir: bool,
):
    keep_mask = color_importance > color_comp.importance_include

    print(
        f"color keep: {keep_mask.float().mean()*100:.2f}%"
    )

    vq_mask_c = ~keep_mask

    # remove zero sh component
    if color_compress_non_dir:
        n_sh_coefs = gaussians.get_features.shape[1]
        color_features = gaussians.get_features.detach().flatten(-2)
    else:
        n_sh_coefs = gaussians.get_features.shape[1] - 1
        color_features = gaussians.get_features[:, 1:].detach().flatten(-2)
    if vq_mask_c.any():
        print("compressing color...")
        color_codebook, color_vq_indices = vq_features(
            color_features[vq_mask_c],
            color_importance[vq_mask_c],
            color_comp.codebook_size,
            color_comp.batch_size,
            color_comp.steps,
        )
    else:
        color_codebook = torch.empty(
            (0, color_features.shape[-1]), device=color_features.device
        )
        color_vq_indices = torch.empty(
            (0,), device=color_features.device, dtype=torch.long
        )

    all_features = color_features
    compressed_features, indices = join_features(
        all_features, keep_mask, color_codebook, color_vq_indices
    )

    gaussians.set_color_indexed(compressed_features.reshape(-1, n_sh_coefs, 3), indices)

#def compress_covariance(
#    gaussians: GaussianModel,
#    gaussian_importance: torch.Tensor,
#    gaussian_comp: CompressionSettings,
#):
#
#    keep_mask_g = gaussian_importance > gaussian_comp.importance_include
#
#    vq_mask_g = ~keep_mask_g
#
#    print(f"gaussians keep: {keep_mask_g.float().mean()*100:.2f}%")
#
#    covariance = gaussians.get_normalized_covariance(strip_sym=True).detach()
#
#    if vq_mask_g.any():
#        print("compressing gaussian splats...")
#        cov_codebook, cov_vq_indices = vq_features(
#            covariance[vq_mask_g],
#            gaussian_importance[vq_mask_g],
#            gaussian_comp.codebook_size,
#            gaussian_comp.batch_size,
#            gaussian_comp.steps,
#            scale_normalize=True,
#        )
#    else:
#        cov_codebook = torch.empty(
#            (0, covariance.shape[1], 1), device=covariance.device
#        )
#        cov_vq_indices = torch.empty((0,), device=covariance.device, dtype=torch.long)
#
#    compressed_cov, cov_indices = join_features(
#        covariance,
#        keep_mask_g,
#        cov_codebook,
#        cov_vq_indices,
#    )
#
#    rot_vq, scale_vq = extract_rot_scale(to_full_cov(compressed_cov))
#
#    gaussians.set_gaussian_indexed(
#        rot_vq.to(compressed_cov.device),
#        scale_vq.to(compressed_cov.device),
#        cov_indices,
#    )

def compress_covariance(
    gaussians: GaussianModel,
    gaussian_importance: torch.Tensor,
    gaussian_comp: CompressionSettings,
):

    # Identity mapping means we don't need to filter with importance
    keep_mask_g = gaussian_importance > gaussian_comp.importance_include
    vq_mask_g = ~keep_mask_g

    print(f"gaussians keep: {keep_mask_g.float().mean()*100:.2f}%")

    # Get the covariance (without stripping symmetry)
    covariance = gaussians.get_normalized_covariance(strip_sym=True).detach()

    # We are no longer performing compression (VQ), so just retain all gaussians
    compressed_cov = covariance

    # Create an identity mapping where each Gaussian `i` maps to rotation `i` and scale `i`
    # `cov_indices` is now a simple range of indices from 0 to number of gaussians
    cov_indices = torch.arange(compressed_cov.shape[0], device=compressed_cov.device)

    # Extract rotation and scale directly from the full covariance matrix
    rot_vq, scale_vq = extract_rot_scale(to_full_cov(compressed_cov))

    # Set the indexed gaussians using identity mapping
    gaussians.set_gaussian_indexed(
        rot_vq.to(compressed_cov.device),
        scale_vq.to(compressed_cov.device),
        cov_indices,  # Identity mapping: index points to itself
    )



def compress_gaussians(
    gaussians: GaussianModel,
    color_importance: torch.Tensor,
    gaussian_importance: torch.Tensor,
    color_comp: Optional[CompressionSettings],
    gaussian_comp: Optional[CompressionSettings],
    color_compress_non_dir: bool,
    prune_threshold:float=0.,
):
    with torch.no_grad():
        if prune_threshold >= 0:
            non_prune_mask = color_importance > prune_threshold
            print(f"prune: {(1-non_prune_mask.float().mean())*100:.2f}%")
            gaussians.mask_splats(non_prune_mask)
            gaussian_importance = gaussian_importance[non_prune_mask]
            color_importance = color_importance[non_prune_mask]
        
        if color_comp is not None:
            compress_color(
                gaussians,
                color_importance,
                color_comp,
                color_compress_non_dir,
            )
        if gaussian_comp is not None:
            compress_covariance(
                gaussians,
                gaussian_importance,
                gaussian_comp,
            )

In [22]:

def accumulate_gradients(grad_storage, model):
    for name, param in model.named_parameters():
        if param.grad is not None:
            if name not in grad_storage:
                grad_storage[name] = {'gradients': []}
            grad_storage[name]['gradients'].append(param.grad.abs().clone().detach())

def calculate_statistics(grad_storage):
    gradients_mean = {}
    gradients_median = {}

    for name, data in grad_storage.items():
        stacked_grads = torch.stack(data['gradients'])
        gradients_mean[name] = stacked_grads.mean(dim=0).mean().item()
        gradients_median[name] = stacked_grads.median(dim=0).values.median().item()

    return gradients_mean, gradients_median


def finetune(scene: Scene, dataset, opt, comp, pipe, testing_iterations, debug_from):
    prepare_output_and_logger(comp.output_vq, dataset)

    first_iter = scene.loaded_iter
    max_iter = first_iter + comp.finetune_iterations

    bg_color = [1, 1, 1] if dataset.white_background else [0, 0, 0]
    background = torch.tensor(bg_color, dtype=torch.float32, device="cuda")

    iter_start = torch.cuda.Event(enable_timing=True)
    iter_end = torch.cuda.Event(enable_timing=True)

    scene.gaussians.training_setup(opt)
    scene.gaussians.update_learning_rate(first_iter)

    viewpoint_stack = None
    ema_loss_for_log = 0.0
    progress_bar = tqdm(range(first_iter, max_iter), desc="Training progress", file=sys.stdout)
    first_iter += 1
    psnr_track = []
    loss_track = []
    ssim_track = []
    grad_storage = {}
    log_interval = 200
    gradients_log = []

    for iteration in range(first_iter, max_iter + 1):
        iter_start.record()

        # Pick a random Camera
        if not viewpoint_stack:
            viewpoint_stack = scene.getTrainCameras().copy()
        viewpoint_cam = viewpoint_stack.pop(randint(0, len(viewpoint_stack) - 1))

        # Render
        if (iteration - 1) == debug_from:
            pipe.debug = True
        render_pkg = render(viewpoint_cam, scene.gaussians, pipe, background, use_mlp=False)
        image, viewspace_point_tensor, visibility_filter, radii = (
            render_pkg["render"],
            render_pkg["viewspace_points"],
            render_pkg["visibility_filter"],
            render_pkg["radii"],
        )

        # Loss
        gt_image = viewpoint_cam.original_image.cuda()
        Ll1 = l1_loss(image, gt_image)
        loss = (1.0 - opt.lambda_dssim) * Ll1 + opt.lambda_dssim * (
            1.0 - ssim(image, gt_image)
        )
        loss.backward()

        accumulate_gradients(grad_storage, scene.gaussians)

        if iteration % log_interval == 0:
            gradients_mean, gradients_median = calculate_statistics(grad_storage)
            gradients_log.append({
                'iteration': iteration,
                'gradients_mean': gradients_mean,
                'gradients_median': gradients_median
            })
            grad_storage.clear()
            json.dump(gradients_log, open(f"./output/{scene.model_name}/gradient_log.json", 'w+'))

        if iteration == first_iter or iteration % 5 == 0 or True:
            eval_results = {}
            
            test_psnrs = []
            test_losses = []
            test_ssims = []
            for tc in scene.getTestCameras():
                t_img = render(tc, scene.gaussians, pipe, background, use_mlp=False)["render"]
                test_psnrs.append(psnr(t_img, tc.original_image).mean().item())
                test_ssims.append(ssim(t_img, tc.original_image).mean().item())
                ll1_test = l1_loss(t_img, tc.original_image)
                loss_test = (1.0 - opt.lambda_dssim) * ll1_test + opt.lambda_dssim * (1.0 - ssim(t_img, tc.original_image))
                test_losses.append(loss_test.item())
            eval_results['PSNR'] = sum(test_psnrs) / len(test_psnrs)
            eval_results['LOSS'] = sum(test_losses) / len(test_losses)
            eval_results['SSIM'] = sum(test_ssims) / len(test_ssims)

            psnr_track.append(eval_results['PSNR'])
            json.dump(psnr_track, open(f"./output/{scene.model_name}/diff_idx_psnr.json", 'w+'))
            loss_track.append(eval_results["LOSS"])
            json.dump(loss_track, open(f"./output/{scene.model_name}/diff_idx_loss.json", 'w+'))
            ssim_track.append(eval_results['SSIM'])
            json.dump(loss_track, open(f"./output/{scene.model_name}/diff_idx_ssim.json", 'w+'))

        iter_end.record()
        scene.gaussians.update_learning_rate(iteration)


        with torch.no_grad():
            # Progress bar
            ema_loss_for_log = 0.4 * loss.item() + 0.6 * ema_loss_for_log
            if iteration % 10 == 0:
                progress_bar.set_postfix({"Loss": f"{ema_loss_for_log:.{7}f}"})
                progress_bar.update(10)
                print(f"Iteration: {iteration}, loss: {loss}")
            if iteration == max_iter:
                progress_bar.close()

            # Optimizer step
            if iteration < max_iter:
                scene.gaussians.optimizer.step()
                scene.gaussians.optimizer.zero_grad()


def prepare_output_and_logger(output_folder, args):
    if not output_folder:
        if os.getenv("OAR_JOB_ID"):
            unique_str = os.getenv("OAR_JOB_ID")
        else:
            unique_str = str(uuid.uuid4())
        output_folder = os.path.join("./output/", unique_str[0:10])

    # Set up output folder
    print("Output folder: {}".format(output_folder))
    os.makedirs(output_folder, exist_ok=True)
    with open(os.path.join(output_folder, "cfg_args"), "w") as cfg_log_f:
        cfg_log_f.write(str(Namespace(**vars(args))))


In [23]:

def calc_importance(
    gaussians: GaussianModel, scene, pipeline_params
) -> Tuple[torch.Tensor, torch.Tensor]:
    scaling = gaussians.scaling_qa(
        gaussians.scaling_activation(gaussians._scaling.detach())
    )
    cov3d = gaussians.covariance_activation(
        scaling, 1.0, gaussians.get_rotation.detach(), True
    ).requires_grad_(True)
    scaling_factor = gaussians.scaling_factor_activation(
        gaussians.scaling_factor_qa(gaussians._scaling_factor.detach())
    )
    
    h1 = gaussians._features_dc.register_hook(lambda grad: grad.abs())
    h2 = gaussians._features_rest.register_hook(lambda grad: grad.abs())
    h3 = cov3d.register_hook(lambda grad: grad.abs())
    background = torch.tensor([0.0, 0.0, 0.0], dtype=torch.float32, device="cuda")

    gaussians._features_dc.grad = None
    gaussians._features_rest.grad = None
    num_pixels = 0
    for camera in tqdm(scene.getTrainCameras(), desc="Calculating sensitivity"):
        cov3d_scaled = cov3d * scaling_factor.square()
        rendering = render(
            camera,
            gaussians,
            pipeline_params,
            background,
            clamp_color=False,
            cov3d=cov3d_scaled,
        )["render"]
        loss = rendering.sum()
        loss.backward()
        num_pixels += rendering.shape[1]*rendering.shape[2]

    importance = torch.cat(
        [gaussians._features_dc.grad, gaussians._features_rest.grad],
        1,
    ).flatten(-2)/num_pixels
    cov_grad = cov3d.grad/num_pixels
    h1.remove()
    h2.remove()
    h3.remove()
    torch.cuda.empty_cache()
    return importance.detach(), cov_grad.detach()

def initialize_gaussians(model_params, comp_params):
    gaussians = GaussianModel(model_params.sh_degree if model_params.sh_degree else 3, quantization=True)
    scene = Scene(model_params, gaussians, load_iteration="", shuffle=True)
    return gaussians, scene

def initial_compress(gaussians, scene, model_params, pipeline_params, optim_params, comp_params):
    print("Setting up compressed model...")
    color_importance, gaussian_sensitivity = calc_importance(gaussians, scene, pipeline_params)
    with torch.no_grad():
        color_importance_n = color_importance.amax(-1)
        gaussian_importance_n = gaussian_sensitivity.amax(-1)
        torch.cuda.empty_cache()

        color_compression_settings = CompressionSettings(
            codebook_size=comp_params.color_codebook_size,
            importance_prune=comp_params.color_importance_prune,
            importance_include=comp_params.color_importance_include,
            steps=int(comp_params.color_cluster_iterations),
            decay=comp_params.color_decay,
            batch_size=comp_params.color_batch_size,
        )

        gaussian_compression_settings = CompressionSettings(
            codebook_size=comp_params.gaussian_codebook_size,
            importance_prune=None,
            importance_include=comp_params.gaussian_importance_include,
            steps=int(comp_params.gaussian_cluster_iterations),
            decay=comp_params.gaussian_decay,
            batch_size=comp_params.gaussian_batch_size,
        )
        
        compress_gaussians(
            gaussians,
            color_importance_n,
            gaussian_importance_n,
            color_compression_settings if not comp_params.not_compress_color else None,
            gaussian_compression_settings
            if not comp_params.not_compress_gaussians
            else None,
            comp_params.color_compress_non_dir,
            prune_threshold=comp_params.prune_threshold,
        )

    gc.collect()
    torch.cuda.empty_cache()
    os.makedirs(comp_params.output_vq, exist_ok=True)

    model_params.model_path = comp_params.output_vq

    return gaussians, scene

def render_and_eval(
    gaussians: GaussianModel,
    scene: Scene,
    model_params: ModelParams,
    pipeline_params: PipelineParams,
) -> Dict[str, float]:
    with torch.no_grad():
        ssims = []
        psnrs = []
        lpipss = []

        views = scene.getTestCameras()

        bg_color = [1, 1, 1] if model_params.white_background else [0, 0, 0]
        background = torch.tensor(bg_color, dtype=torch.float32, device="cuda")

        for i, view in enumerate(tqdm(views, desc="Rendering progress")):
            rendering = render(view, gaussians, pipeline_params, background)[
                "render"
            ].unsqueeze(0)
            gt = view.original_image[0:3, :, :].unsqueeze(0)

            ssims.append(ssim(rendering, gt))
            psnrs.append(psnr(rendering, gt))
            lpipss.append(lpips(rendering, gt, net_type="vgg"))
            gc.collect()
            torch.cuda.empty_cache()
            torchvision.utils.save_image(gt, f"renders/{scene.model_name}/gt_{i}.png") 
            torchvision.utils.save_image(rendering, f"renders/{scene.model_name}/render_{i}.png") 
        
        return {
            "SSIM": torch.tensor(ssims).mean().item(),
            "PSNR": torch.tensor(psnrs).mean().item(),
            "LPIPS": torch.tensor(lpipss).mean().item(),
        }



In [24]:
# Code that produces stderr output
parser = ArgumentParser(description="Compression script parameters")
model = ModelParams(parser, sentinel=True)
model.data_device = "cuda"
pipeline = PipelineParams(parser)
op = OptimizationParams(parser)
comp = CompressionParams(parser)
parser.add_argument('--ip', type=str, default="127.0.0.1")
parser.add_argument('--port', type=int, default=6009)
parser.add_argument('--debug_from', type=int, default=-1)
parser.add_argument('--detect_anomaly', action='store_true', default=False)
parser.add_argument("--save_iterations", nargs="+", type=int, default=[7_000, 30_000])
parser.add_argument("--quiet", action="store_true")
parser.add_argument('--disable_viewer', action='store_true', default=False)
parser.add_argument("--checkpoint_iterations", nargs="+", type=int, default=[])
sys.argv = ['asdfadsf.py', "-s", 'a', '-r', '4', '--eval']
args = parser.parse_args()
args.save_iterations.append(args.iterations)

model_params = model.extract(args)
optim_params = op.extract(args)
pipeline_params = pipeline.extract(args)
comp_params = comp.extract(args)

model_params.source_path = '/scratch/sankeerth/bonsai'
model_params.model_path = './output/bonsai'
comp_params.output_vq = './output/bonsai'


comp_params.finetune_iterations = 0
if not model_params.sh_degree:
    model_params.sh_degree = 3

gaussians, scene = initialize_gaussians(model_params, comp_params)

if not os.path.exists(f"renders/{scene.model_name}/"):
    os.mkdir(f"renders/{scene.model_name}/")
if not os.path.exists(f"renders/{scene.model_name}/training"):
    os.mkdir(f"renders/{scene.model_name}/training")

gaussians, scene = initial_compress(gaussians, scene, model_params, pipeline_params, optim_params, comp_params)

comp_params.finetune_iterations = 30_000
scene.loaded_iter = 0

finetune(
    scene,
    model_params,
    optim_params,
    comp_params,
    pipeline_params,
    testing_iterations=[-1],
    debug_from = -1
)

iteration = comp_params.finetune_iterations
out_file = path.join(
    comp_params.output_vq,
    f"point_cloud/iteration_{iteration}/point_cloud.npz",
)
gaussians.save_npz(out_file, sort_morton=not comp_params.not_sort_morton)
file_size = os.path.getsize(out_file) / 1024**2
print(f"saved vq finetuned model to {out_file}")

# eval model
print("evaluating...")
metrics = render_and_eval(gaussians, scene, model_params, pipeline_params)
metrics["size"] = file_size
print(metrics)
with open(f"{comp_params.output_vq}/results.json","w") as f:
    json.dump({f"ours_{iteration}":metrics},f,indent=4)


MODEL NAME:  bonsai
Reading camera 292/292
Loading Training Cameras
device() received an invalid combination of arguments - got (NoneType), but expected one of:
 * (torch.device device)
      didn't match because some of the arguments have invalid types: (!NoneType!)
 * (str type, int index)

[Warning] Custom device None failed, fallback to default cuda device
device() received an invalid combination of arguments - got (NoneType), but expected one of:
 * (torch.device device)
      didn't match because some of the arguments have invalid types: (!NoneType!)
 * (str type, int index)

[Warning] Custom device None failed, fallback to default cuda device
device() received an invalid combination of arguments - got (NoneType), but expected one of:
 * (torch.device device)
      didn't match because some of the arguments have invalid types: (!NoneType!)
 * (str type, int index)

[Warning] Custom device None failed, fallback to default cuda device
device() received an invalid combination of arg















Calculating sensitivity: 100%|██████████| 255/255 [00:01<00:00, 181.27it/s]


prune: 0.00%
color keep: 5.17%
compressing color...







































































































100%|██████████| 100/100 [00:14<00:00,  6.92it/s]


Training progress:   0%|          | 40/30000 [03:27<43:05:28,  5.18s/it, Loss=0.2276484]
calculating indices took 0.1038210391998291 seconds 
gaussians keep: 1.56%
Output folder: ./output/bonsai
Training progress:   1%|          | 160/30000 [00:45<2:22:18,  3.49it/s, Loss=0.1045241]Iteration: 160, loss: 0.0913914367556572
